# Import everything

In [1]:
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm.notebook import tqdm
import math
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras_tqdm import TQDMNotebookCallback
from tensorboard.plugins.hparams import api as hp
from livelossplot.tf_keras import PlotLossesCallback

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave
from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from skimage.transform import resize
from timeit import default_timer as timer
from datetime import timedelta
import json
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks, butter
from scipy.integrate import simps
from scipy.io import loadmat
from numpy import trapz
import gzip
import scipy.fftpack

Using TensorFlow backend.
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


# Read files from disk

In [2]:
def read_trials_from_raw(filename, time_start, time_end, columns):
    x = loadmat(filename)
    x = x['EEGData']
    trials = []
    time = np.linspace(time_start, time_end, len(x[0, :, 0]))
    for trial_num in range(x.shape[2]):
        trial = np.transpose(x[:, :, trial_num])
        trial = pd.DataFrame(data=trial, columns=columns)
        trial['time'] = time
        trials.append(trial)
    return trials

def read_trials_from_hjorth(filename, time_start, time_end):
    x = loadmat(filename)
    mat_trials = x['dat'][0][0][3][0]
    trials = []
    time = np.linspace(time_start, time_end, len(mat_trials[0][0]))
    for mat_trial in mat_trials:
        trials.append(pd.DataFrame({'C3': mat_trial[0], 'C4': mat_trial[1], 'time': time}))
    return trials

# Power and phase calculations

In [3]:
def blackman_harris_filter(channel, time, cutoffs, fs, numtaps):
    b = signal.firwin(numtaps, cutoffs, window='blackmanharris', fs=fs)
    filtered = signal.lfilter(b, 1, channel)
    delay = 0.5 * (numtaps - 1) / fs
    df = pd.DataFrame({
        'time': time-delay,
        'channel': filtered
    })
    return df

def butter_bandpass_filter(data, lowcut, highcut, fs, btype, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=btype)
    y = signal.lfilter(b, a, data)
    return y

def get_phase(channel, time, band, fs, filter_type, start_time_ms, stop_time_ms, blackmann_harris_ntaps):
    if filter_type=='butter':
        df_filtered = pd.DataFrame({'channel': butter_bandpass_filter(channel, band[0], band[1], fs, btype='bandpass', order=3), 'time': time})
    else:
        df_filtered = blackman_harris_filter(channel, time, [0.000001, band[0]], fs, numtaps=blackmann_harris_ntaps)
        df_filtered = blackman_harris_filter(df_filtered['channel'].values, df_filtered['time'].values, band[1], fs, numtaps=blackmann_harris_ntaps)
    
    hilb = signal.hilbert(df_filtered[(df_filtered['time'] > start_time_ms) & (df_filtered['time'] < stop_time_ms)]['channel'])
    phase = np.angle(hilb, deg=True)
    df_phase = pd.DataFrame(phase, columns=['phase'])
    df_phase['time'] = df_filtered[(df_filtered['time'] > start_time_ms) & (df_filtered['time'] < stop_time_ms)]['time'].values
    return df_phase

def get_power(channel, time, fs, crop_start_millis, crop_end_millis, method, filter_type, resample_fs, line_noise_band, blackmann_harris_ntaps):
    # Crop.
    channel = channel[(time > crop_start_millis) & (time < crop_end_millis)].values
    time = time[(time > crop_start_millis) & (time < crop_end_millis)].values
    
    # Resample
    if resample_fs is None:
        resampled = channel
        resampled_time = time
    else:
        ts = time[1] - time[0]
        secs = len(channel) * ts
        secs = secs/1000
        resampled = signal.resample(channel, int(secs*resample_fs))
        resampled_time = np.linspace(time[0], time[-1], len(resampled))
        fs = resample_fs

    # Remove line nosie.
    if filter_type == 'butter':
        resampled = butter_bandpass_filter(resampled, line_noise_band[0], line_noise_band[1], fs, 'bandstop', order=4)
        resampled_time = time
    else:
        df_filtered = blackman_harris_filter(resampled, resampled_time, line_noise_band, fs, numtaps=blackmann_harris_ntaps)
        resampled = df_filtered['channel']
        resampled_time = df_filtered['time']
    
    # PSD.
    if method == 'welch':
        # Welch method
        freq, power = signal.welch(resampled, fs, nperseg=4*fs, nfft=fs)
        df_power = pd.DataFrame({'freq': freq, 'power': power})
    elif method == 'fft':
        # FFT method
        T = 1/fs
        N = fs//2
        yf = scipy.fftpack.fft(resampled, n=N)
        yf = 2 / (N/2) * np.abs(yf[:N//2])
        xf = np.linspace(0, 1/(2*T), N // 2)
        df_power = pd.DataFrame({'freq': xf, 'power': yf})
    elif method == 'pburg':
        # Burgs method
        order = min(len(resampled)-2, int(fs/4))
        p = pburg(resampled, order, sampling=fs, NFFT=fs)
        power = p.psd
        freq = np.linspace(0, fs/2, len(power))
        df_power = pd.DataFrame({'freq': freq, 'power': power})

    df_power = df_power[df_power['freq'] < fs/2]
    df_power['power'] = 10 * np.log10(df_power['power'])
    return df_power

In [4]:
raw_files = sorted(glob.glob('data/dataset2/raw/*.mat'))
raw_hjorth_files = sorted(glob.glob('data/dataset2/raw-hjorth/*.mat'))
clean_files = sorted(glob.glob('data/dataset2/clean/*.mat'))
clean_hjorth_files = sorted(glob.glob('data/dataset2/clean-hjorth/*.mat'))
print('{}, {}, {}, {}'.format(len(raw_files), len(raw_hjorth_files), len(clean_files), len(clean_hjorth_files)))

122, 122, 122, 122


# Diagnose #1

In [5]:
# ind = 100
# trial_num = 20
# print("{} - {}".format(ind, raw_files[ind]))

# trials = read_trials_from_raw(raw_files[ind], -900, 0, ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd'])
# # trials = read_trials_from_raw(clean_files[ind], -900, 0, ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd'])
# df_trial = trials[trial_num]

# channel = df_trial['C3']
# time = df_trial['time']

# fs = 256
# band = [8, 12]
# blackmann_harris_ntaps = 51
# line_noise_band = [48, 52]
# time_start = -150


# i = 0
# f, axs = plt.subplots(1,3,figsize=(16,2))
# axs[i].plot(time, channel)
# axs[i].set_title('Waveform')
# i = i + 1

# freq, power = signal.welch(channel, fs, nperseg=4*fs, nfft=fs)
# df_power = pd.DataFrame({'freq': freq, 'power': power})
# df_power = df_power[df_power['freq'] < 80]
# axs[i].plot(df_power['freq'], df_power['power'])
# axs[i].set_title('PSD - Welch')
# i = i + 1

# order = min(len(channel)-2, int(fs/4))
# p = pburg(channel, 64, sampling=fs, NFFT=fs)
# power = p.psd
# freq = np.linspace(0, fs/2, len(power))
# df_power = pd.DataFrame({'freq': freq, 'power': power})
# df_power = df_power[df_power['freq'] < 80]
# df_power = df_power[df_power['freq'] >3]
# axs[i].plot(df_power['freq'], df_power['power'])
# axs[i].set_title('PSD - Burg')
# i = i + 1
# print(order)

# Diagnose #2

In [6]:
# ind = 100
# trial_num = 20
# print("{} - {}".format(ind, raw_files[ind]))

# trials = read_trials_from_raw(raw_files[ind], -900, 0, ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd'])
# df_trial = trials[trial_num]

# channel = df_trial['C3']
# time = df_trial['time']

# fs = 256
# band = [8, 12]
# blackmann_harris_ntaps = 51
# line_noise_band = [48, 52]
# time_start = -150

# for filter_type in ['butter', 'blackmann']:
#     i = 0
#     f, axs = plt.subplots(1,6,figsize=(16,2))
#     axs[i].plot(time, channel)
#     i = i + 1
    
#     freq, power = signal.welch(channel, fs, nperseg=4*fs, nfft=fs)
#     df_power = pd.DataFrame({'freq': freq, 'power': power})
#     df_power = df_power[df_power['freq'] < 80]
#     axs[i].plot(df_power['freq'], df_power['power'])
#     i = i + 1

#     df_phase = get_phase(channel, time, band=band, fs=fs, filter_type=filter_type, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=blackmann_harris_ntaps)
#     axs[i].plot(df_phase['time'], df_phase['phase'])
#     axs[i].set_title("Phase - {}".format(filter_type))
#     i = i + 1

#     for method in ['pburg', 'welch', 'fft']:
#         df_power = get_power(channel, time, fs, crop_start_millis=time_start, crop_end_millis=-1, method=method, filter_type=filter_type, resample_fs=None, line_noise_band=line_noise_band, blackmann_harris_ntaps=blackmann_harris_ntaps)
#         df_power = df_power[df_power['freq'] < 130]
#         axs[i].plot(df_power['freq'], df_power['power'])
#         axs[i].set_title("Power - {} - {}".format(filter_type, method))
#         i = i + 1
#     plt.tight_layout()
#     plt.show()

# Process trials

In [8]:
def process_subject(raw_path, hjorth_path, clean_raw_path, clean_hjorth_path):
    df_powers = []
    df_phases = []
    subject = raw_path.split('/')[3].split('.')[0]
    
    trials_raw = read_trials_from_raw(raw_path, -900, 0, ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd'])
    trials_hjorth = read_trials_from_hjorth(hjorth_path, -900, 0)
    trials_clean_raw = read_trials_from_raw(clean_raw_path, -900, 0, ['AF1', 'AF2', 'AF7', 'AF8', 'AFZ', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPZ', 'CZ', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FCZ', 'FP1', 'FP2', 'FPZ', 'FT7', 'FT8', 'FZ', 'O1', 'O2', 'OZ', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'PO1', 'PO2', 'PO7', 'PO8', 'POZ', 'PZ', 'T7', 'T8', 'TP7', 'TP8', 'X', 'Y', 'nd'])
    trials_clean_hjorth = read_trials_from_hjorth(clean_hjorth_path, -900, 0)
    
    df_trialnums = pd.read_csv('data/dataset2/raw/trialnum-{}.csv'.format(subject))
    trialnums = df_trialnums['Var1'].apply(lambda x: int(x.split('-')[1].split('.')[0])).values
    df_sub = pd.read_csv('data/dataset2/subjects/' + subject + '.csv')
    
    for trial_num in range(len(trials_raw)):
        if df_sub[df_sub['trial'] == trialnums[trial_num]].shape[0] == 0:
            continue
        trial_raw = trials_raw[trial_num]
        trial_raw_hjorth = trials_hjorth[trial_num]
        trial_clean = trials_clean_raw[trial_num]
        trial_clean_hjorth = trials_clean_hjorth[trial_num]
        
        trial_clean_avg = trial_clean.copy()
        trial_clean_avg['C3'] = trial_clean_avg[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
        trial_raw_avg = trial_raw.copy()
        trial_raw_avg['C3'] = trial_raw_avg[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
        
        power_row = {
            'sub': subject,
            'trial': trial_num,
            'alcoholic': df_sub.iloc[0]['alcoholic'],
            'condition': df_sub[df_sub['trial'] == trialnums[trial_num]].iloc[0]['condition']
        }
        
        for eeg_type, artifact_removed, df_trial in zip(['Raw', 'Hjorth', 'Raw', 'Hjorth', 'Average', 'Average'], [False, False, True, True, True, False], [trial_raw, trial_raw_hjorth, trial_clean, trial_clean_hjorth, trial_clean_avg, trial_raw_avg]):
            for filter_code, filter_name in zip(['butter', 'blackmannharris'], ['Butterworth', 'Blackmann-Harris']):
                for time, ntaps in zip([-750, -250], [101, 51]):
                    for method_code, method_name in zip(['pburg', 'welch', 'fft'], ['Burg', 'Welch', 'FFT']):

                        try:
                            df_power = get_power(df_trial['C3'], df_trial['time'], fs=256, crop_start_millis=time, crop_end_millis=-1, method=method_code, filter_type=filter_code, resample_fs=None, line_noise_band=[48, 52], blackmann_harris_ntaps=ntaps)
                        except:
                            continue

                        for band, fc1, fc2 in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [4, 8, 13, 30], [8, 12, 30, 80]):
                            power = df_power[df_power['freq'] >= fc1]
                            power = power[power['freq'] < fc2]
                            power = power['power'].mean(axis=0)
                            row = power_row.copy()
                            row['ArtifactRemoved'] = artifact_removed
                            row['EEG'] = eeg_type
                            row['Filter'] = filter_name
                            row['Time'] = time
                            row['Method'] = method_name
                            row['Band'] = band
                            row['Power'] = power
                            df_powers.append(row)
                for band, fc1, fc2 in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [4, 8, 13, 30], [8, 12, 30, 80]):
                    phase = get_phase(df_trial['C3'], df_trial['time'], [fc1, fc2], filter_type=filter_code, start_time_ms=-750, stop_time_ms=-1, fs=256, blackmann_harris_ntaps=101)
                    phase = phase.iloc[-1]['phase'] + 180
                    row = power_row.copy()
                    row['ArtifactRemoved'] = artifact_removed
                    row['EEG'] = eeg_type
                    row['Filter'] = filter_name
                    row['Band'] = band
                    row['Phase'] = phase
                    df_phases.append(row)
    return (df_powers, df_phases)

In [9]:
# Run analysis
num_cores = multiprocessing.cpu_count() - 2
results = Parallel(n_jobs=num_cores)(delayed(process_subject)(raw, hjorth, clean, clean_hjorth) for raw, hjorth, clean, clean_hjorth in tqdm(zip(raw_files, raw_hjorth_files, clean_files, clean_hjorth_files), total=len(raw_files)))

In [10]:
df_power = []
df_phase = []
for powers, phases in results:
    for row in powers:
        df_power.append(row)
    for row in phases:
        df_phase.append(row)
df_power = pd.DataFrame(df_power)
df_phase = pd.DataFrame(df_phase)
df_power.head()

,ArtifactRemoved,Band,EEG,Filter,Method,Power,Time,alcoholic,condition,sub,trial
0,False,Theta,Raw,Butterworth,Burg,-7.396500,-750,True,S1 obj,co2a0000364,0
1,False,Mu,Raw,Butterworth,Burg,-6.664451,-750,True,S1 obj,co2a0000364,0
2,False,Beta,Raw,Butterworth,Burg,-3.637847,-750,True,S1 obj,co2a0000364,0
3,False,Gamma,Raw,Butterworth,Burg,-4.195108,-750,True,S1 obj,co2a0000364,0
4,False,Theta,Raw,Butterworth,Welch,-10.963867,-750,True,S1 obj,co2a0000364,0


In [11]:
def_values_power = {
    'ArtifactRemoved': False,
    'EEG': 'Raw', 
    'Filter': 'Butterworth', 
    'Time': -750, 
    'Method': 'Welch'
}
def_values_phase = {
    'ArtifactRemoved': False,
    'EEG': 'Raw',
    'Filter': 'Butterworth', 
}

In [12]:
df_power = df_power[df_power['alcoholic'] == False]
df_power = df_power[df_power['condition'] == 'S1 obj']

df_power_new = []
keys = list(def_values_power.keys())
for sub in tqdm(df_power['sub'].unique()):
    df_sub = df_power[df_power['sub'] == sub]
    for val1 in df_power[keys[0]].unique():
        for val2 in df_power[keys[1]].unique():
            for val3 in df_power[keys[2]].unique():
                for val4 in df_power[keys[3]].unique():
                    for val5 in df_power[keys[4]].unique():
                        for val6 in df_power['Band'].unique():
                            df_power2 = df_sub[df_sub['Band'] == val6]
                            df_power2 = df_power2[df_power2[keys[4]] == val5]
                            df_power2 = df_power2[df_power2[keys[3]] == val4]
                            df_power2 = df_power2[df_power2[keys[2]] == val3]
                            df_power2 = df_power2[df_power2[keys[1]] == val2]
                            df_power2 = df_power2[df_power2[keys[0]] == val1]
                            i = 1
                            for idx, row in df_power2.iterrows():
                                row['trial_abs'] = i
                                df_power_new.append(row)
                                i = i + 1
df_power_new = pd.DataFrame(df_power_new)

In [13]:
df_phase = df_phase[df_phase['alcoholic'] == False]
df_phase = df_phase[df_phase['condition'] == 'S1 obj']

df_phase_new = []
keys = list(def_values_phase.keys())
for sub in tqdm(df_phase['sub'].unique()):
    df_sub = df_phase[df_phase['sub'] == sub]
    for val1 in df_phase[keys[0]].unique():
        for val2 in df_phase[keys[1]].unique():
            for val3 in df_phase[keys[2]].unique():
                for val4 in df_phase['Band'].unique():
                    df_phase2 = df_sub[df_sub[keys[0]] == val1]
                    df_phase2 = df_phase2[df_phase2[keys[1]] == val2]
                    df_phase2 = df_phase2[df_phase2[keys[2]] == val3]
                    df_phase2 = df_phase2[df_phase2['Band'] == val4]
                    i = 1
                    for idx, row in df_phase2.iterrows():
                        row['trial_abs'] = i
                        df_phase_new.append(row)
                        i = i + 1
df_phase_new = pd.DataFrame(df_phase_new)

In [14]:
df_power_new.to_csv('157-d2-power-v2.csv')
df_phase_new.to_csv('157-d2-phase-v2.csv')